# ROI-based univariate analyses
Natalia Vélez, April 2022

In [ ]:
%matplotlib inline

import os, sys
import pandas as pd
import numpy as np
from nilearn import image,plotting,masking
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join as opj
from scipy.io import loadmat
from scipy import stats

sys.path.append('..')
from utils import gsearch, str_extract

sns.set_style('white')
sns.set_context('talk')

## Setup

Valid subjects:

In [ ]:
subjects = np.loadtxt('../1_preprocessing/outputs/valid_participants.txt', dtype=int)
print(subjects)

Project directory:

In [ ]:
data_dir = '/n/gershman_ncf/Lab/natalia_teaching/BIDS_data/derivatives'
model_name = 'task-teaching_model-parametricnonortho'

Find ACC:

In [ ]:
acc_file = '/ncf/gershman/User/nvelezalicea/fmri_analysis/roi_library/fmriprep_space/bilateral_ACCg.nii.gz'
os.path.exists(acc_file)

Find functional ROI files:

In [ ]:
roi_files = gsearch(data_dir, 'roi_picker', 'sub-*', 'func', '*.nii.gz')
roi_files.sort()

print(f'Found {len(roi_files)} ROI files')
print(*roi_files[:10]+['...'], sep='\n')

Find contrast files:

In [ ]:
con_files = gsearch(data_dir, 'glm', 'sub-*', 'func', model_name, 'con*')
con_files.sort()

print(f'Found {len(con_files)} contrast files')
print(*con_files[:10]+['...'], sep='\n')

Load contrast names:

In [ ]:
con_file = opj(data_dir, 'glm', 'group', model_name, 'contrasts.mat')
con_data = loadmat(con_file)
contrasts_raw = con_data['contrasts']
contrasts = [c[0].replace('+', '') for c in contrasts_raw[0]] # all a hacky way of reading mat files
print(f'Found {len(contrasts)} contrasts')
print(*contrasts, sep='\n')

## GLM 1: Full model with model-based regressors

### First-level contrasts

Main loop: Extract data from all ROIs

In [ ]:
# Initialize output
roi_list = []

# Iterate over subjects and contrasts
for sub in subjects:
    for con_idx,con_name in enumerate(contrasts):
        
        # Format subject, contrast numbers nicely
        sub_id = 'sub-%02d' % sub
        con_id = 'con_%04d' % (con_idx+1)

        # Filter out contrast, ROI files
        sub_con = [f for f in con_files if sub_id in f and con_id in f]
        sub_con = sub_con[0] # should be a unique file

        sub_roi = [f for f in roi_files if sub_id in f]
        sub_roi.append(acc_file) # add anatomical ACC ROI
        sub_roi.sort() # list of files
                
        # Extract average t from each ROI
        for roi in sub_roi:
            roi_name = str_extract('(?<=desc-)[A-Za-z]+|ACC', roi)            
            masked_t = masking.apply_mask(sub_con, roi)
            mean_t = masked_t.mean()
            
            roi_list.append((sub, con_name, roi_name, mean_t))

In [ ]:
# put it all together
roi_df = pd.DataFrame(roi_list, columns=['subject', 'contrast', 'roi', 'beta'])
roi_df['roi'] = roi_df.roi.astype('category')
roi_df = roi_df.sort_values(by=['subject', 'contrast', 'roi']).reset_index(drop=True)
print(roi_df.shape)
roi_df.head(8)

Do one-sample t-tests with Bonferroni correction:

In [ ]:
t_list = []

# do one-sample t-tests on average ROI betas
for name,group in roi_df.groupby(['roi', 'contrast']):
    # find CI (for plotting)
    ci_lo, ci_hi = sns.utils.ci(sns.algorithms.bootstrap(group.beta))
    group_mean = group.beta.mean()
    
    res = stats.ttest_1samp(group.beta, 0)
    t_list.append(name+(group_mean, res.statistic, res.pvalue, ci_lo, ci_hi))
    
t_df = (
    pd.DataFrame(t_list, columns=['roi', 'contrast', 'avg', 'statistic', 'pvalue', 'ci_lo', 'ci_hi'])
    .sort_values(by=['contrast', 'roi'])
    .reset_index(drop=True)
)
t_df['roi'] = t_df['roi'].astype('category')

# mark ROIs that survive Bonferroni correction
p_thresh = 0.05/(t_df.roi.nunique()+1)
t_df['sig'] = np.where(t_df.pvalue < p_thresh, '*', '') # extra vars for plotting
t_df['sig_y'] = np.where(t_df.avg >= 0, t_df.ci_hi+.1, t_df.ci_lo-.3)
t_df['sig_x'] = t_df.roi.cat.codes-.11

t_df

#### 1) Posterior probability of true hypothesis

Plot results:

In [ ]:
pTrue_df = roi_df[roi_df.contrast == 'pTrue']
fig,ax = plt.subplots(figsize=(8,4))
sns.barplot(data=pTrue_df, x='roi', y='beta', fc='#007fcf', ax=ax)

pTrue_sig = t_df[t_df.contrast == 'pTrue']
for _, row in pTrue_sig.iterrows():
    ax.text(row.sig_x, row.sig_y, row.sig)

ax.set(xlabel='', ylabel='Beta', title = r'Belief in true hypothesis: $P_L(h_T|d)$', ylim=(
    pTrue_sig.ci_lo.min()*1.25,
    pTrue_sig.ci_hi.max()*1.25
))

#### 2) Belief update

In [ ]:
KL_df = roi_df[roi_df.contrast == 'KL']
fig,ax = plt.subplots(figsize=(8,4))
sns.barplot(data=KL_df, x='roi', y='beta', fc='#ff9b0f', ax=ax)

KL_sig = t_df[t_df.contrast == 'KL']
for _, row in KL_sig.iterrows():
    ax.text(row.sig_x, row.sig_y, row.sig)

ax.set(xlabel='', ylabel='Beta', title = 'Belief update (KL divergence)', ylim=(
    KL_sig.ci_lo.min()*1.25,
    KL_sig.ci_hi.max()*1.25
))

### (Exploratory) Second-level contrasts

Load participant-specific predictors:

In [ ]:
sub_models = pd.read_csv('../2_behavioral/outputs/second_level_model_regressors.csv')
sub_models.head()

Merge with subject-specific betas:

In [ ]:
second_level = roi_df.merge(sub_models)
second_level.head()

#### 1) Posterior probability of true hypothesis

Plot second-order relationships:

In [ ]:
second_pTrue = second_level[second_level.contrast == 'pTrue']

g = sns.lmplot(data=second_pTrue, x="logBF", y="beta", col="roi", col_wrap=4)
g.set_titles('{col_name}')
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle(r'Belief in true hypothesis: $P_L(h_T|d)$')

Correlation tests:

In [ ]:
pTrue_corr_list = []
for name,group in second_pTrue.groupby('roi'):
    r,p = stats.pearsonr(group.logBF, group.beta)
    pTrue_corr_list.append((name, r, p))
    
pTrue_corr = pd.DataFrame(pTrue_corr_list, columns=['roi', 'correlation', 'pvalue'])
pTrue_corr

#### 2) Belief update

Plot second-order relationships:

In [ ]:
second_KL = second_level[second_level.contrast == 'KL']

g = sns.lmplot(data=second_KL, x="logBF", y="beta", col="roi", col_wrap=4, scatter_kws={'color':'#ff9b0f'}, line_kws={'color':'#ff9b0f'})
g.set_titles('{col_name}')
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle('Belief update (KL divergence)')

Correlation tests:

In [ ]:
KL_corr_list = []
for name,group in second_KL.groupby('roi'):
    r,p = stats.pearsonr(group.logBF, group.beta)
    KL_corr_list.append((name, r, p))
    
KL_corr = pd.DataFrame(KL_corr_list, columns=['roi', 'correlation', 'pvalue'])
KL_corr

## GLM 2: Parametric regressors derived from student responses

### First-level contrasts

### (Exploratory) Second-level contrasts

### (Exploratory) Comparing to student performance

## GLM 3-4: Model-based regressors estimated in separate GLMs

### First-level contrasts

### (Exploratory) Second-level contrasts